In [1]:
#!/usr/bin/env PYTHONHASHSEED=1234 python3

# Copyright 2014-2019 Brett Slatkin, Pearson Education Inc.
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     http://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Reproduce book environment
import random
random.seed(1234)

import logging
from pprint import pprint
from sys import stdout as STDOUT

# Write all output to a temporary directory
import atexit
import gc
import io
import os
import tempfile

TEST_DIR = tempfile.TemporaryDirectory()
atexit.register(TEST_DIR.cleanup)

# Make sure Windows processes exit cleanly
OLD_CWD = os.getcwd()
atexit.register(lambda: os.chdir(OLD_CWD))
os.chdir(TEST_DIR.name)

def close_open_files():
    everything = gc.get_objects()
    for obj in everything:
        if isinstance(obj, io.IOBase):
            obj.close()

atexit.register(close_open_files)

<function __main__.close_open_files()>

In [2]:
# Example 1
stock = {
    'nails': 125,
    'screws': 35,
    'wingnuts': 8,
    'washers': 24,
}

order = ['screws', 'wingnuts', 'clips']

def get_batches(count, size):
    return count // size

result = {}
for name in order:
  count = stock.get(name, 0)
  batches = get_batches(count, 8)
  if batches:
    result[name] = batches

print(result)

{'screws': 4, 'wingnuts': 1}


In [3]:
# Example 2
found = {name: get_batches(stock.get(name, 0), 8)
         for name in order
         if get_batches(stock.get(name, 0), 8)}
print(found)

{'screws': 4, 'wingnuts': 1}


In [4]:
# Example 3
has_bug = {name: get_batches(stock.get(name, 0), 4)
           for name in order
           if get_batches(stock.get(name, 0), 8)}

print('Expected:', found)
print('Found:   ', has_bug)

Expected: {'screws': 4, 'wingnuts': 1}
Found:    {'screws': 8, 'wingnuts': 2}


In [5]:
# Example 4
found = {name: batches for name in order
         if (batches := get_batches(stock.get(name, 0), 8))}
assert found == {'screws': 4, 'wingnuts': 1}, found

In [6]:
# Example 5
try:
    result = {name: (tenth := count // 10)
              for name, count in stock.items() if tenth > 0}
except:
    logging.exception('Expected')
else:
    assert False

ERROR:root:Expected
Traceback (most recent call last):
  File "<ipython-input-6-45f4c1cf2ea5>", line 3, in <module>
    result = {name: (tenth := count // 10)
  File "<ipython-input-6-45f4c1cf2ea5>", line 4, in <dictcomp>
    for name, count in stock.items() if tenth > 0}
NameError: name 'tenth' is not defined


In [7]:
# Example 6
result = {name: tenth for name, count in stock.items()
          if (tenth := count // 10) > 0}
print(result)

{'nails': 12, 'screws': 3, 'washers': 2}


In [8]:
# Example 7
half = [(last := count // 2) for count in stock.values()]
print(f'Last item of {half} is {last}')

Last item of [62, 17, 4, 12] is 12


In [9]:
# Example 8
for count in stock.values():  # Leaks loop variable
    pass
print(f'Last item of {list(stock.values())} is {count}')

Last item of [125, 35, 8, 24] is 24


In [10]:
# Example 9
try:
    del count
    half = [count // 2 for count in stock.values()]
    print(half)   # Works
    print(count)  # Exception because loop variable didn't leak
except:
    logging.exception('Expected')
else:
    assert False

ERROR:root:Expected
Traceback (most recent call last):
  File "<ipython-input-10-44ef610c87ae>", line 6, in <module>
    print(count)  # Exception because loop variable didn't leak
NameError: name 'count' is not defined
[62, 17, 4, 12]


In [12]:
# Example 10
found = ((name, batches) for name in order
         if (batches := get_batches(stock.get(name, 0), 8)))
print(next(found))
print(next(found))

<generator object <genexpr> at 0x7e0aab4c4970>
('screws', 4)
('wingnuts', 1)
